In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from lib import preprocess, sentiment_analysis

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
spark_session = SparkSession.builder \
    .master("spark://192.168.2.47:7077") \
    .appName("test") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
    .config("spark.cores.max", 12) \
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/19 22:07:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/19 22:07:11 WARN StandaloneSchedulerBackend: Dynamic allocation enabled without spark.executor.cores explicitly set, you may get more executors allocated than expected. It's recommended to set spark.executor.cores explicitly. Please check SPARK-30299 for more details.


In [4]:
df = spark_session.read.option("mode", "DROPMALFORMED").json("hdfs://192.168.2.47:9000/data-project/reddit_50k.json")

In [5]:
df_prep_top10_sr = preprocess(df)

In [6]:
df_prep_top10_sr.printSchema()

root
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- clean_text: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
df_prep_top10_sr.show(10)

[Stage 2:>                  (0 + 0) / 1][Stage 3:>                  (0 + 0) / 1]

In [ ]:
df_vader = sentiment_analysis(df_prep_top10_sr)

In [ ]:
df_vader.printSchema()

In [8]:
df_vader.show(10)

25/03/19 21:33:39 ERROR TaskSetManager: Task 0 in stage 2.0 failed 4 times; aborting job


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/ubuntu/spark-3.5.5-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/home/ubuntu/spark-3.5.5-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/home/ubuntu/spark-3.5.5-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/home/ubuntu/spark-3.5.5-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/home/ubuntu/spark-3.5.5-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return s

In [ ]:
df_avg = df_vader.groupBy("subreddit") \
                 .agg(F.round(F.avg("sentiment_score"), 4).alias("avg_sentiment_score")) \
                 .limit(10)

In [ ]:
df_avg.show()